In [ ]:
from itertools import islice
import math

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

In [ ]:
# The path to a gzipped CSV containing forex data from Polygon.io.
DATA_PATH = "../../data/filtered/2025-01-01_filtered.csv"

In [ ]:
df = pd.read_csv(DATA_PATH)

df

In [ ]:
df.head(20)

In [ ]:
# Construct a directed, weighted graph in which the nodes
# are currencies and the edge weights are exchange rates.

N_TIMESTAMPS = 1

graph = dict()

grouped = df.groupby("participant_timestamp")
for timestamp, batch in islice(grouped, N_TIMESTAMPS):
    for index, row in batch.iterrows():
        currency1, currency2 = row["ticker"][2:].split("-")
        bid_price = row["bid_price"]
    
        if currency1 not in graph:
            graph[currency1] = dict()
        graph[currency1][currency2] = bid_price

In [ ]:
# Visualize the forex graph.

from math import log

COI = []

G = nx.DiGraph()

for currency1, rates in graph.items():
    for currency2, rate in rates.items():
        if len(COI) == 0 or currency1 in COI and currency2 in COI:
            G.add_edge(currency1, currency2, weight=rate)

pos = nx.nx_pydot.graphviz_layout(G, prog="neato")
# pos = nx.circular_layout(G)
edge_labels = nx.get_edge_attributes(G, "weight")

plt.figure(figsize=(10, 5))
# nx.draw(G, pos, with_labels=True, node_size=1000, node_color="#FF7777")
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw(G, pos, with_labels=True, node_size=1000, node_color="#FF7777", connectionstyle="arc3,rad=0.2")
# nx.draw_networkx_edges(G, pos, edgelist=[("SGD", "CHF"), ("CHF", "USD"), ("USD", "SGD")], edge_color="red", connectionstyle="arc3,rad=0.2")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, connectionstyle="arc3,rad=0.2")

plt.savefig("graph.png", dpi=300)
plt.show()